In [1]:
%cd ~
import pyfits,glob,time,scipy,sys
import matplotlib
matplotlib.use("PDF")
from pylab import *
thedir='devel/treerings/output/'
%cd $thedir


/Users/cslage
[Errno 2] No such file or directory: 'devel/treerings/output/'
/Users/cslage


/usr/local/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
zfilelist=sort(glob.glob(thedir+'tr_?.fits'))
zfilelist

array([ '/Users/cslage/Research/LSST/code/galsim-20oct17/GalSim/devel/treerings/output/tr_1.fits',
       '/Users/cslage/Research/LSST/code/galsim-20oct17/GalSim/devel/treerings/output/tr_2.fits'], 
      dtype='|S87')

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [4]:
class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.imax=zeros([nstamps])


def BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 0
    xcoomax = 800
    ycoomin = 0
    ycoomax = 800
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    img=pyfits.getdata(fitsfilename,0) 
    catname=fitsfilename[:-5]+'_SEX.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = img[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xint + xoff#xcoord - xint - 1.0
            spotlist.yoffset[n] = yint + yoff#ycoord - yint - 1.0     
                    
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist



In [5]:
nx = 9
ny = 9
numspots = 100
results = []
spotlists=[]
segmentnumber = 0

for fitsfilename in zfilelist:
    print fitsfilename
    
    spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny,0.2,2.0)
    print "nstamps = %d"%spotlist.nstamps

    spotlists.append(spotlist)


/Users/cslage/Research/LSST/code/galsim-20oct17/GalSim/devel/treerings/output/tr_1.fits
nstamps = 100
/Users/cslage/Research/LSST/code/galsim-20oct17/GalSim/devel/treerings/output/tr_2.fits
nstamps = 100


In [6]:
figure()
subplot(1,1,1,aspect=1.0)
title("Tree Ring Test on GalSim Generated Spots", fontsize=18)
Mult = 10.0
for i in range(spotlists[0].nstamps):
    x = spotlists[0].xoffset[i]
    y = spotlists[0].yoffset[i] 
    dx = Mult*(spotlists[0].xoffset[i]-spotlists[1].xoffset[i])
    dy = Mult*(spotlists[0].yoffset[i]-spotlists[1].yoffset[i])
    arrow(x,y,dx,dy, color='red', lw = 1.0, width = 0.10)

plot([170,270],[150,250], color='blue', lw = 2)
ylim(0,400)
xlim(0,400)
xlabel('X (Pixels)')
ylabel('Y (Pixels)')
text(410,200 ,"Tree ring Amplitude", fontsize = 8)
text(410,185 ,"=1.0 pixels.", fontsize = 8)
text(410,155 ,"Plotted shift", fontsize = 8)
text(410,140 ,"is multiplied by 10.", fontsize = 8)
text(410,110 ,"Blue bar is", fontsize = 8)
text(410,95 ,"period of 141.3 pixels.", fontsize = 8)
text(410,65 ,"Tree Ring Center", fontsize = 8)
text(410,50 ,"is at (-1000,-1000).", fontsize = 8)
savefig(thedir+"Spot_Shift.pdf")